# Part 2 - Training the model generate text jointly from emojis and text

Following the success in getting the model to generate text based on training it on twitter data, we're going to modify the model to generate text based on two inputs - the emoji data and the text data.

While the text will still be fed character-by-character into the LSTM, we're going to add an additional Dense input, which is just an n_emoji dimensional vector, where n_emoji is the number of possible emojis that we have sufficient data for (ie., more than 1000 training examples). The emojis will be one-hot encoded.

First, we need to modify the data_load_utils files so that `convert_to_xy` gives us both one-hot encodings (text and emoji).

In [1]:
import numpy as np
import pandas as pd
import data_load_utils as util
from math import ceil

from importlib import reload
util = reload (util)

# for cpu and memory profiling
#%load_ext line_profiler
#%load_ext memory_profiler




Let's filter the data down to only emojis with more than 1000 training examples, and clean up the text by filtering out twitter handles, as we did before

In [2]:
tweets = util.filter_tweets_min_count(util.read_tweet_data('data/emojis_homemade.csv'), min_count=1000)

tweets['text'] = util.filter_text_for_handles(tweets['text'])

In [3]:
tweets.shape

(445129, 2)

First, let's get a list of all the emojis present in the tweets['emojis'] data series.

In [4]:
MAX_TWEET_LENGTH = 160
WINDOW_SIZE = 64
STEP = 3

samples_per_tweet = int(ceil((MAX_TWEET_LENGTH - WINDOW_SIZE) / STEP)) # 32
tweets_per_batch = 2 #64
samples_per_batch = samples_per_tweet * tweets_per_batch # 2048

chars_univ, chars_univ_idx = util.get_universal_chars_list()

emoji_series = tweets['emoji']
emojis, emoji_idx = util.get_emojis_list(emoji_series)

In [5]:
# for prototyping
#TRAIN_SIZE = 2**12
#DEV_SIZE = 2**10

TRAIN_SIZE = 2**18 # 32,768  try 131072 = 2**17 for production
DEV_SIZE = 2**12   # 8192  try 8192 = 2**13 for production

n_train_batches = TRAIN_SIZE / tweets_per_batch
n_dev_batches = DEV_SIZE / tweets_per_batch

tweets_train = tweets.iloc[0:TRAIN_SIZE] # 8192 = 2**13
tweets_dev = tweets.iloc[TRAIN_SIZE:TRAIN_SIZE+DEV_SIZE] # 2048 = 2**11

In [6]:
# 64 tweets x 32 samples per tweet = 2048 training examples per batch
train_generator = util.convert_tweet_to_xy_generator(tweets_train, length=MAX_TWEET_LENGTH, \
                                                     window_size=WINDOW_SIZE,step=STEP, \
                                                     batch_size=tweets_per_batch, emoji_set=emojis)

dev_generator = util.convert_tweet_to_xy_generator(tweets_dev, length=MAX_TWEET_LENGTH, \
                                                   window_size=WINDOW_SIZE,step=STEP, \
                                                   batch_size=tweets_per_batch, emoji_set = emojis)

# Building a network
Intially, let's try generating tweets by training a network on just the tweet data. Once we have an idea how well we can get a network to generate tweets (remember, character by character), we'll compare it to a network that learns to generate tweets by predicting the next chracter jointly from the preceding text and an overall emoji. (remember, this dataset is tweets that all contain exactly one emoji).

## Simple network - a single LSTM into a Dense softmax classifier.

In [7]:
# Establish the shapes of the inputs/outputs
([x_text, x_emoji], y) = next(train_generator)
print ("x_text shape:", x_text.shape)
print ("x_emoji shape:", x_emoji.shape)
print ("y shape:", y.shape)

x_text shape: (64, 64, 93)
x_emoji shape: (64, 111)
y shape: (64, 93)


x_text: 2048 x 64 x 93 (batch_size x window_size x len(chars_univ))
x_emoji 2048 x 112 (batch_size x chars_univ)
y 2048 x 93 (batch_size x chars_univ)

In [8]:
import keras
from keras import layers, Input
from keras.models import Model
from keras import callbacks

Using TensorFlow backend.


In [56]:
text_input = Input(shape=(WINDOW_SIZE, len(chars_univ)), dtype='float32', name='text_input')
lstm = layers.LSTM(256, name='text_lstm')(text_input)

emoji_input = Input(shape=(len(emojis),),
                    dtype='float32',
                    name='emoji_input')
emoji_dense = layers.Dense (50, activation='relu')(emoji_input)

concatenate = layers.concatenate([lstm, emoji_dense], name='concatenate')

output = layers.Dense(len(chars_univ), name='output',
                      activation='softmax')(concatenate)

model = Model([text_input, emoji_input], output)
model.compile(optimizer = keras.optimizers.RMSprop(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])


model.summary()


# can use this saved file for transfer learning
# model = keras.models.load_model("models/tweet_gen_model-0.776.hdf5") # 256 LSTM units, ~30 epochs training  # 

#model = keras.models.Sequential()
#model.add(layers.LSTM(256, input_shape=(WINDOW_SIZE, len(chars_univ)))) # was 128 units
#model.add(layers.Dense(len(chars_univ), activation='softmax'))

# loss function - targets are one-hot encoded



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 64, 93)       0                                            
__________________________________________________________________________________________________
emoji_input (InputLayer)        (None, 112)          0                                            
__________________________________________________________________________________________________
text_lstm (LSTM)                (None, 256)          358400      text_input[0][0]                 
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 50)           5650        emoji_input[0][0]                
__________________________________________________________________________________________________
concatenat

Let's see if we can use transfer learning to save a bit of time, by re-using the trained weights from the text-only model.

In [57]:
tweet_only_model = keras.models.load_model("models/text_gen_model-train250k-0.830.hdf5")
# 256 LSTM units, ~30 epochs training

tweet_model_weights = tweet_only_model.get_layer(index=0).get_weights()
current_model_lstm_layer = model.get_layer(name='text_lstm')
current_model_lstm_layer.set_weights(tweet_model_weights)


## Training the model and sampling from it using a standard character-by-character method
1. Draw a probability distribution for the next character
2. Reweight the distribution using a temperature parameter
3. Sample the next character at random using the reweighted distribution
4. Add the new character at the end of the available list

In [15]:
def sample (preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## train the model, generate text
Use a range of temeratures after every epoch

In [59]:
tweets.iloc[0]['text'][0:10]

'RT [VID] 1'

In [9]:
model = keras.models.load_model("models/text_emoji_joint_gen_model-0.840.hdf5") # resume progress, 10+2+1 epoch completed

In [62]:
import random
import sys

n_seed_chars = 64 # number of characters to use as a seed for text generation

model.optimizer.lr.assign(0.001) # to reset the learning rate if running additional training

checkpoint = callbacks.ModelCheckpoint(filepath='text_emoji_joint_gen_model-{loss:.3f}.hdf5', 
                                       verbose=1, 
                                       save_best_only=True)

# train for 60 epochs
for epoch in range (1, 60):
    print ('epoch', epoch)

    # fit the model for one iteration
    model.fit_generator (train_generator,
                         steps_per_epoch=n_train_batches, # 64 x 32 = batches of 2048
                         epochs=1,
                         validation_data=dev_generator, 
                         validation_steps=n_dev_batches,
                         callbacks=[checkpoint],
                         verbose=1,
                         use_multiprocessing=True, # run the generator in a separate thread
                         )

    # select a text seed at random
    seed_tweet = tweets.iloc[random.randint(0, len(tweets))]
    seed_text = util.pad_text(seed_tweet['text'][0:n_seed_chars], n_seed_chars)
    generated_text = seed_text
    #one-hot encode the emoji
    emoji_one_hot = util.get_emoji_bool_array(seed_tweet['emoji'], emoji_idx)
    print ('--- Generating with seed: "' + generated_text + '" | ' + seed_tweet['emoji'])

    
    # try a range of sampling temperatures
    for temperature in [0.3, 0.5, 0.8, 1.0]:
        generated_text = seed_text
        print ('--------- temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range (MAX_TWEET_LENGTH - n_seed_chars):
            # one-hot encode the characters generated so far
            sampled = np.zeros((1, WINDOW_SIZE, len(chars_univ)))
            for t, char in enumerate (generated_text):
                sampled[0, t, chars_univ_idx[char]] = 1

            # sample the next character
            preds = model.predict([sampled, emoji_one_hot], verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars_univ[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

        print ("\n")    

epoch 1
Epoch 1/1
64/64 [==============================] - 35s 543ms/step - loss: 0.3892 - acc: 0.9402 - val_loss: 1.2701 - val_acc: 0.7197

Epoch 00001: val_loss improved from inf to 1.27012, saving model to text_emoji_joint_gen_model-0.389.hdf5
--- Generating with seed: "RT I was fighting with my nigga one day in my house and I told h" | 😭
--------- temperature: 0.3
RT I was fighting with my nigga one day in my house and I told his this ag this way this                                                                        

--------- temperature: 0.5
RT I was fighting with my nigga one day in my house and I told his the retwott the my how thas I ave this hat by the soud this seostiouf  hatisid this  happy th

--------- temperature: 0.8
RT I was fighting with my nigga one day in my house and I told has #bythasingranenot   Re watt as you and readiti they 4o real lit rettr  stintre the reagestzmy

--------- temperature: 1.0
RT I was fighting with my nigga one day in my house and I told

     RT Any day all day, Ill choose this https://t.co/mYfoGBmW2RsaSlasSelaGe wored  leangalle we won thas thitkid happy.  reepe.  lell spolle  is camphitre wos 

--------- temperature: 0.8
     RT Any day all day, Ill choose this https://t.co/mYfoGBmW2R #lassipaylisicagmape and copepates Oo happy Roddillied thingso.                                

--------- temperature: 1.0
     RT Any day all day, Ill choose this https://t.co/mYfoGBmW2RlackidgIIl AR 40 idancmenichPO wdNObe myching patslest angps you wow s as  leck simentt the myen

epoch 17
Epoch 1/1
64/64 [==============================] - 34s 532ms/step - loss: 0.1415 - acc: 0.9844 - val_loss: 1.3997 - val_acc: 0.7148

Epoch 00001: val_loss did not improve from 1.26825
--- Generating with seed: "I love impractical jokers so much! I wish I had the confidence a" | 😂
--------- temperature: 0.3
I love impractical jokers so much! I wish I had the confidence and sond the matill who  no dont won  loding with this shotts follo  all with th

Process ForkPoolWorker-137:
Process ForkPoolWorker-136:
Traceback (most recent call last):
  File "/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/nickdbn/anaconda3/envs/deeplearn

KeyboardInterrupt: 

In [10]:
import tensorflow as tf
import random, sys
def generate_tweet(seed_text, seed_emoji, emoji_idx, window_size=WINDOW_SIZE,
		   length=MAX_TWEET_LENGTH, temperatures=[0.3, 0.5, 0.8, 1.0],
		   random_seed = None):

    if random_seed:
        np.random.seed(random_seed) # np.random.seed needed by tf 
        tf.set_random_seed(random_seed)
        

    n_seed_chars = window_size
    #first, pad out the seed_text with whitespace     
    seed_text = util.pad_text(seed_text, window_size)
    #one-hot encode the emoji
    emoji_one_hot = util.get_emoji_bool_array(seed_emoji, emoji_idx)
    print ('--- Generating with seed: "' + seed_text + '" [' + str(len(seed_text)) + ']')

    # try a range of sampling temperatures
    for temperature in temperatures:
        generated_text = seed_text
        print ('--------- temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range (length - n_seed_chars):
            # one-hot encode the characters generated so far
            sampled = np.zeros((1, WINDOW_SIZE, len(chars_univ)))
            for t, char in enumerate (generated_text):
                sampled[0, t, chars_univ_idx[char]] = 1

            # sample the next character
            preds = model.predict([sampled, emoji_one_hot], verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars_univ[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

        print ("\n")    



In [11]:
def get_seed_text(tweets, seed_length=64, random_seed = None):
    if random_seed: np.random.seed(random_seed)
        
    seed_tweet = tweets.iloc[np.random.randint(0, len(tweets))]
    seed_text = seed_tweet['text'][0:seed_length]
    seed_emoji = seed_tweet['emoji']
    
    print (seed_text + "\n" + seed_tweet['text'] + "|" + seed_emoji)
    
    return seed_text, seed_emoji


In [12]:
seed_text, seed_emoji = get_seed_text(tweets, seed_length=64, random_seed=5)

Omg youre sweet, thank you!
Omg youre sweet, thank you!|☺


In [23]:
generate_tweet (seed_text, '💓', emoji_idx, length=200, temperatures=[0.8, 0.9, 1.0, 1.1, 1.2])

--- Generating with seed: "                                     Omg youre sweet, thank you!" [64]
--------- temperature: 0.8
                                     Omg youre sweet, thank you!! I still be much for an important moment and shaie jumping for this for words, bettee  #lovestanding # #wins   # # # # # # #JIMIN# # # 

--------- temperature: 0.9
                                     Omg youre sweet, thank you!!! and I did walking nothing out in an a spssss and shes left the lovery star private ruming her family  where the lucky                

--------- temperature: 1.0
                                     Omg youre sweet, thank you! And hake fucking Go done is not feel positivel!! Thanks to like that name I can come for my nigga  and she is more!!  @likeaselandim #s

--------- temperature: 1.1
                                     Omg youre sweet, thank you!  Gillurinura waste   A heall spagiuiting  &amp; wanting the ball a Happy thing abu  beforeah I have been a hair clinch Nide

In [14]:
emoji_idx

{'©': 0,
 '‼': 1,
 '▶': 2,
 '☹': 3,
 '☺': 4,
 '♥': 5,
 '⚠': 6,
 '⚡': 7,
 '✅': 8,
 '✈': 9,
 '✊': 10,
 '✌': 11,
 '✔': 12,
 '✨': 13,
 '❗': 14,
 '❣': 15,
 '❤': 16,
 '➡': 17,
 '⭐': 18,
 '🌟': 19,
 '🌹': 20,
 '🎃': 21,
 '🎉': 22,
 '🎶': 23,
 '🏆': 24,
 '👀': 25,
 '👇': 26,
 '👉': 27,
 '👊': 28,
 '👌': 29,
 '👍': 30,
 '👏': 31,
 '👑': 32,
 '💀': 33,
 '💋': 34,
 '💓': 35,
 '💔': 36,
 '💕': 37,
 '💖': 38,
 '💗': 39,
 '💘': 40,
 '💙': 41,
 '💚': 42,
 '💛': 43,
 '💜': 44,
 '💞': 45,
 '💥': 46,
 '💦': 47,
 '💪': 48,
 '💫': 49,
 '💯': 50,
 '📍': 51,
 '📷': 52,
 '📸': 53,
 '📽': 54,
 '🔗': 55,
 '🔥': 56,
 '🔴': 57,
 '🖤': 58,
 '🗣': 59,
 '😀': 60,
 '😁': 61,
 '😂': 62,
 '😄': 63,
 '😅': 64,
 '😆': 65,
 '😇': 66,
 '😈': 67,
 '😉': 68,
 '😊': 69,
 '😋': 70,
 '😌': 71,
 '😍': 72,
 '😎': 73,
 '😏': 74,
 '😐': 75,
 '😑': 76,
 '😒': 77,
 '😔': 78,
 '😕': 79,
 '😘': 80,
 '😜': 81,
 '😞': 82,
 '😡': 83,
 '😢': 84,
 '😤': 85,
 '😩': 86,
 '😪': 87,
 '😫': 88,
 '😬': 89,
 '😭': 90,
 '😱': 91,
 '😳': 92,
 '😴': 93,
 '🙂': 94,
 '🙃': 95,
 '🙄': 96,
 '🙈': 97,
 '🙌': 98,
 '🙏': 99,
 '🚨': 100,